In [1]:
# Wczytanie bibliotek

%pylab inline
import pandas as pd
import os
import keras
import joblib
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Baza danych

images_dir = "C:/wikiart/"
df = pd.read_csv('C:/wikiart/wclasses.csv')
training_imgs = list(images_dir + df['file'])
df['filepath'] = training_imgs

df['century']=df['artist']

for i in df.index:
  filename = os.path.basename(df['file'][i])
  parts = filename.split('-')
  yearonly = parts[len(parts)-1].split('.')[0]
  try:
    yearonly=int(yearonly) 
  except ValueError:
    yearonly=None

  try:
    if 1900<yearonly<2001:
      df['century'][i]=20
    elif 1800<yearonly<1901:
      df['century'][i]=19
    elif 1700<yearonly<1801:
      df['century'][i]=18
    elif 1600<yearonly<1701:
      df['century'][i]=17
    elif 1500<yearonly<1601:
      df['century'][i]=16
    elif 1400<yearonly<1501:
      df['century'][i]=15
    else:
      df['century'][i]=-1
  except TypeError:
    df['century'][i]=-1
    
df['Artist_name'] = df.file.str.split("/",expand=True)[1]
df['Artist_name'] = df.Artist_name.str.split("_",expand=True)[0]

style_number = list(range(27))
style_number = [x+140 for x in style_number]

genre_number = list(range(11))
genre_number = [x+129 for x in genre_number]

century_number1=[-1]
century_number = list(range(6))
century_number = [x+15 for x in century_number]
century_number = century_number1 + century_number

columns_style = ['Ekspresjonizm Abstrakcyjny', 'Taszyzm', 'Kubizm Analityczny', 'Art Noveau', 'Barok', 'Malarstwo Barwnych Płaszczyzn', 'Realizm Współczesny', 'Kubizm', 'Wczesny Renesans', 'Ekspresjonizm', 'Fowizm', 'Wysoki Renesans', 'Impresjonizm', 'Późny Renesans', 'Minimalizm', 'Prymitywizm', 'Nowy Realizm', 'Północny Renesans', 'Puentylizm', 'Pop Art', 'Postimpresjonizm', 'Realizm', 'Rokoko', 'Romantyzm', 'Symbolizm', 'Kubizm Syntetyczny', 'Ukiyo-e']
columns_genre = ['Abstrakcja', 'Pejzaż Miejski', 'Malarstwo Rodzajowe', 'Ilustracja', 'Krajobraz', 'Akt', 'Portret', 'Malarstwo Religijne', 'Szkic', 'Martwa Natura','Nieznany Gatunek']
columns_century = ['Nieznane Stulecie','XV', 'XVI', 'XVII', 'XVIII', 'XIX', 'XX']

style_dict = dict(zip(style_number, columns_style))
genre_dict = dict(zip(genre_number, columns_genre))
century_dict = dict(zip(century_number, columns_century))

df['Artist_name'] = df['Artist_name'].str.replace('-', ' ').values
df['Artist_name'] = df.Artist_name.str.title()
df['Style_name'] = df['style'].map(style_dict)
df['Genre_name'] = df['genre'].map(genre_dict)
df['Century_name'] = df['century'].map(century_dict)



<ipython-input-2-d22f369446fe>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['century'][i]=19
<ipython-input-2-d22f369446fe>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['century'][i]=17
<ipython-input-2-d22f369446fe>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['century'][i]=-1
<ipython-input-2-d22f369446fe>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the c

In [3]:
# Zbiory testowe i walidacyjne

# Identyfikacja wieku

df_century1 = df[df.century != -1]
train1, test_century1 = train_test_split(df_century1, test_size = 0.15, random_state = 21, stratify = df_century1['century'])
train, val_century1 = train_test_split(train1, test_size = 0.177, random_state = 21, stratify = train1['century'])

# Identyfikacja stylu

train1, test_style1 = train_test_split(df, test_size = 0.15, random_state = 21, stratify = df['style'])
train, val_style1 = train_test_split(train1, test_size = 0.15, random_state = 21, stratify = train1['style'])

# Identyfikacja artysty

training_set_count = df.groupby(['Artist_name']).size().reset_index(name='count')
training_set_count = training_set_count[training_set_count['count'] > 250]
training_set_count = training_set_count[training_set_count['count'] != 350]

df_artist1 =  df[df['Artist_name'].isin(training_set_count['Artist_name'])]
train1, test_artist1 = train_test_split(df_artist1, test_size = 0.15, random_state = 21, stratify = df_artist1['Artist_name'])
train, val_artist1 = train_test_split(train1, test_size = 0.177, random_state = 21, stratify = train1['Artist_name'])

# Identyfikacja gatunku

df_genre1 = df[df.genre !=139]
train1, test_genre1 = train_test_split(df_genre1, test_size = 0.15, random_state = 21, stratify = df_genre1['genre'])
train, val_genre1 = train_test_split(train1, test_size = 0.15, random_state = 21, stratify = train1['genre'])

In [4]:
# Mała baza danych

df_mini =  df[df['Artist_name'].isin(training_set_count['Artist_name'])]
df_mini = df_mini[df_mini.genre !=139]
df_mini = df_mini[df_mini.century != -1]
df_mini = df_mini.reset_index()
df_mini.to_csv('C:/MAG/df2/df_mini.csv')

In [5]:
# Złączenie zbiorów testowych i walidacyjnych w celu zapewnienia ciągłości 
# (dzięki tej operacji, te same obrazy zbioru testowego na poziomie niższym będą w zbiorze testowym na poziomie wyższym)

test_artist_togo = pd.merge(test_artist1, df_mini, how='inner', on=['file'])
test_artist_togo.to_csv('C:/MAG/df2/test_artist_togo.csv')

In [6]:
val_artist_togo = pd.merge(val_artist1, df_mini, how='inner', on=['file'])
val_artist_togo.to_csv('C:/MAG/df2/val_artist_togo.csv')

In [7]:
test_genre_togo = pd.merge(test_genre1, df_mini, how='inner', on=['file'])
test_genre_togo.to_csv('C:/MAG/df2/test_genre_togo.csv')

In [8]:
val_genre_togo = pd.merge(val_genre1, df_mini, how='inner', on=['file'])
val_genre_togo.to_csv('C:/MAG/df2/val_genre_togo.csv')

In [9]:
test_style_togo = pd.merge(test_style1, df_mini, how='inner', on=['file'])
test_style_togo.to_csv('C:/MAG/df2/test_style_togo.csv')

In [10]:
val_style_togo = pd.merge(val_style1, df_mini, how='inner', on=['file'])
val_style_togo.to_csv('C:/MAG/df2/val_style_togo.csv')

In [11]:
test_century_togo = pd.merge(test_century1, df_mini, how='inner', on=['file'])
test_century_togo.to_csv('C:/MAG/df2/test_century_togo.csv')

In [12]:
val_century_togo = pd.merge(val_century1, df_mini, how='inner', on=['file'])
val_century_togo.to_csv('C:/MAG/df2/val_century_togo.csv')